In [40]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [41]:
import math
import plotly.express as px
import pandas as pd
import numpy
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
import plotly.graph_objects as go

<IPython.core.display.Javascript object>

# Parameter (können geändert werden)

Messdistanz vor Ort [m]

In [42]:
messung = 1000

<IPython.core.display.Javascript object>

maximal gewünschter Fehler [mm]

In [43]:
max_error = 10

<IPython.core.display.Javascript object>

## Variablen

Erdradius in m für Schmiegekugel in der Schweiz

In [44]:
R = 6378815.90365

<IPython.core.display.Javascript object>

Punkte in 1903+/LV95 mit Name, N, E, Höhe ü. Meer (LN02) [m].
Ein neuer Punkt kann einfach eingefügt werden.

In [45]:
points = {
    "Cadenazzo": {"Name": "Cadenazzo", "N": 1112721.7, "E": 2715783.3, "Height": 208.0},
    "Bern": {"Name": "Bern", "N": 1200000, "E": 2600000, "Height": 553.6},
    "Muttenz": {"Name": "Muttenz", "N": 1264806.3, "E": 2615766.2, "Height": 280.7},
    "Chessel": {"Name": "Chessel", "N": 1133230, "E": 2558128.3, "Height": 378.6},
    "Melano": {"Name": "Melano", "N": 1086300, "E": 2719998.3, "Height": 281.7},
    "Kraftwerk Untere Enge": {
        "Name": "Kraftwerk Untere Enge",
        "N": 1201114,
        "E": 2600259.5,
        "Height": 500.0,
    },
    "Bottinghofen am Bodensee": {
        "Name": "Bottinghofen am Bodensee",
        "N": 1278503.30,
        "E": 2733173.3,
        "Height": 400,
    },
    "Zürich HB": {"Name": "Zürich HB", "N": 1248258, "E": 2682858.2, "Height": 406.2},
    "Boncourt": {"Name": "Boncourt", "N": 1260400, "E": 2568683.3, "Height": 444.4},
    "CERN": {"Name": "CERN", "N": 1121229, "E": 2493320, "Height": 437.9},
    "Neuchatel": {"Name": "Neuchatel", "N": 1205267.3, "E": 2561781.7, "Height": 479.4},
    "Interlaken Ost": {
        "Name": "Interlaken Ost",
        "N": 1171210.3,
        "E": 2633121.1,
        "Height": 566.7,
    },
    "Campognolo": {"Name": "Campognolo", "N": 1123758.3, "E": 2808405, "Height": 570.7},
    "Bargen, SH": {
        "Name": "Bargen, SH",
        "N": 1294123.3,
        "E": 2687782.5,
        "Height": 606.3,
    },
    "Visp": {"Name": "Visp", "N": 1127065.8, "E": 2634147.9, "Height": 650.9},
    "Langnau": {"Name": "Langnau", "N": 1199011.7, "E": 2626370.8, "Height": 686.4},
    "Vallorbe": {"Name": "Vallorbe", "N": 1174126.7, "E": 2518397.5, "Height": 807},
    "Martina": {"Name": "Martina", "N": 1197083.5, "E": 2830592.5, "Height": 1034},
    "Ste-Croix": {"Name": "Ste-Croix", "N": 1186306.7, "E": 2528490, "Height": 1085.5},
    "Andermatt": {
        "Name": "Andermatt",
        "N": 1165702.5,
        "E": 2688434.1,
        "Height": 1435.7,
    },
    "Monte Generoso": {
        "Name": "Monte Generoso",
        "N": 1087610,
        "E": 2722608.3,
        "Height": 1605.5,
    },
    "Saas Almagell": {
        "Name": "Saas Almagell",
        "N": 1104891.7,
        "E": 2640122.9,
        "Height": 1671,
    },
    "Rigi": {"Name": "Rigi", "N": 1212273.44, "E": 2679520.05, "Height": 1797},
    "Alp Grüm": {"Name": "Alp Grüm", "N": 1139211.2, "E": 2799454.8, "Height": 2090.9},
    "Ofenpass": {"Name": "Ofenpass", "N": 1169362.9, "E": 2818462.9, "Height": 2150.1},
    "Payerne": {"Name": "Payerne", "N": 1186360.8, "E": 2561992.8, "Height": 447.3},
    "Eigergletscher": {
        "Name": "Eigergletscher",
        "N": 1158376.8,
        "E": 2641168.5,
        "Height": 2352,
    },
    "Sphinx": {"Name": "Sphinx", "N": 1155280, "E": 2641911, "Height": 3571},
    "Stausee Mattmark": {
        "Name": "Stausee Mattmark",
        "N": 1099723.3,
        "E": 2639939.2,
        "Height": 2198.4,
    },
    "Wasserstollen Uetliberg": {
        "Name": "Wasserstollen Uetliberg",
        "N": 1245500,
        "E": 2679000,
        "Height": 382,
    },
}

<IPython.core.display.Javascript object>

## Funktionen

Näherungsformeln für die Längenverzerrung infolge Projektion und Höhe.

Umrechnung von N Werten in sogenannte Zivilkoordinaten

In [46]:
def distance_to_Bern(N):
    return N - 1200000

<IPython.core.display.Javascript object>

Horizontaldistanz im Projektionssystem (Karte): Je weiter weg von Bern (Richtung Nord/Süd), desto länger werden Strecken.

In [47]:
def DProj(d0, N):
    return d0 * (1 + distance_to_Bern(N) ** 2 / (2 * R ** 2))

<IPython.core.display.Javascript object>

Eine Strecke dproj in der Realität gemessen entspricht am Bezugshorizont (Bern) einer kürzeren Distanz d0:

In [48]:
def d0(dproj, N):
    return dproj * (1 - distance_to_Bern(N) ** 2 / (2 * R ** 2))

<IPython.core.display.Javascript object>

Horizontaldistanz auf korrekter Höhe (hier will ich messen)

In [49]:
def d_h_uber_Meer(h0, h):
    return h0 * (1 + h / (R + h))

<IPython.core.display.Javascript object>

Horizontaldistanz auf dem Bezugshorizont (Meereshöhe; Karte verwendet diese Höhe). Für Übertrag der Distanz in die Karte brauche ich d_h0

In [50]:
def d_h0(d_h_uber_Meer, h):
    return d_h_uber_Meer * (1 - h / (R + h))

<IPython.core.display.Javascript object>

In [51]:
def massstab(N):
    return distance_to_Bern(N) ** 2 / (2 * R ** 2) + 1

<IPython.core.display.Javascript object>

In [52]:
def height_error_factor(height):
    """height: Mittlere Höhe der Strecke über dem Bezugshorizont."""
    return 1 + height / (R + height)

<IPython.core.display.Javascript object>

In [53]:
def gesamtfehler_von_messung_zu_karte_mm(messung, name, northing, heighting):
    """Eine Messung messung in der Realität wird am Punkt point in einer Karte falsch abgebildet um wieviele mm."""
    fehler_mm = (
        DProj(messung, northing) - messung + d_h0(messung, heighting) - messung
    ) * 1000
    # print(f"{name:>24}: {fehler_mm:> 9.3f} mm")
    return fehler_mm

<IPython.core.display.Javascript object>

In [54]:
def max_distance_allowed_in_m(name, max_error, northing, heighting):
    """Wenn ich maximal den max_error als Fehler tolerieren will, wie lang kann mein Bauwerk auf der gegebenen Höhe sein?"""
    distance_m = abs(
        max_error
        / 1000
        / (distance_to_Bern(northing) ** 2 / (2 * R ** 2) - heighting / (R + heighting))
    )
    # print(f"{name:>24}: {distance_m:>9.3f} m")
    return distance_m

<IPython.core.display.Javascript object>

Für BIM könnte eigentlich die Berücksichtigung der Längenverzerrung reichen: Bei der Übernahme von Grundlagedaten sollten sowohl Längen- als auch Höhenverzerrung berücksichtigt werden (v.A. bei Übernahme aus Papierdaten; bei elektronischen Daten in einem GIS sollten die Metadaten die Erfassung beschreiben). Für das Erstellen eines Modells und den Übertrag in die Realität würde man idealerweise die Geländehöhe definieren und dort im Massstab 1:1 arbeiten. 

In [55]:
def length_error_reality_to_plan_mm(messung, northing):
    return (DProj(messung, northing) - messung) * 1000

<IPython.core.display.Javascript object>

Höhenfehler wegen Erdkrümmung [mm]

In [68]:
h0 = (math.sqrt((R ** 2 + messung ** 2)) - R) * 1000

<IPython.core.display.Javascript object>

## Berechnungen

Gesamtfehler für Messdistanz *messung* sowie Faktoren für jeden Punkt aus der Liste der Schweizer Orte.

In [56]:
# print(f"Für eine gemessene Distanz von {messung} m ergibt sich ein Gesamtfehler von")
for name, values in points.items():
    values.update({"Massstabfaktor": massstab(values["N"])})
    values.update({"Höhenverzerrungsfaktor": height_error_factor(values["Height"])})
    values.update(
        {"Längenfehler [mm]": length_error_reality_to_plan_mm(messung, values["N"])}
    )
    values.update(
        {
            "Gesamtfehler [mm]": gesamtfehler_von_messung_zu_karte_mm(
                messung, name, values["N"], values["Height"]
            )
        }
    )

<IPython.core.display.Javascript object>

Berechne max. Distanz für Gesamtfehler < 10mm

In [57]:
# print(
#    f"Für einen maximalen akzeptablen Fehler von {max_error} mm ist folgende Distanz möglich:"
# )
for name, values in points.items():
    values.update(
        {
            "max. Distanz [m]": max_distance_allowed_in_m(
                name, max_error, values["N"], values["Height"]
            )
        }
    )

<IPython.core.display.Javascript object>

# Visualisierung

### Gesamtfehler [mm] für gegebene Messstrecke in der Realität

Die Grösse des Kreises entspricht der Höhe ü. Meer (LN02) des Punktes.


In [58]:
# fig = px.scatter(data, x="E",y="N",size="max. Distanz [m]", hover_data=["name"], color="Gesamtfehler [mm]")
fig = px.scatter(
    points.values(),
    x="E",
    y="N",
    hover_data=["Name"],
    color="Gesamtfehler [mm]",
    size="Height",
    title=f"Verzerrung [mm] infolge Höhe und Projektion für eine Strecke von {messung}m Länge",
)

fig.show()

<IPython.core.display.Javascript object>

In [59]:
fig = px.scatter(
    points.values(),
    x="E",
    y="N",
    hover_data=["Name"],
    color="Gesamtfehler [mm]",
    size="Height",
    title=f"Verzerrung [mm] infolge Höhe und Projektion für eine Strecke von {messung}m",
    range_x=[2700000, 2740000],
    range_y=[1080000, 1095000],
    text="Name",
)
fig.show()

<IPython.core.display.Javascript object>

### Maximale Distanz [m] für einen gegebenen erlaubten Gesamtfehler

In [60]:
fig = px.scatter(
    points.values(),
    x="E",
    y="N",
    hover_data=["Name"],
    color="max. Distanz [m]",
    title=f"Maximale Distanz [m] für eine akzeptierte Gesamtverzerrung von {max_error} mm",
)

fig.show()

<IPython.core.display.Javascript object>

### Längenverzerrung infolge Projektion für gegebene Messstrecke in der Realität

In [61]:
fig = px.scatter(
    points.values(),
    x="E",
    y="N",
    hover_data=["Name"],
    color="Längenfehler [mm]",
    title=f"Längenverzerrung infolge Projektion für eine gegebene Strecke von {messung} m in der Realität",
)

fig.show()

<IPython.core.display.Javascript object>

### Massstabsfaktor

In [62]:
fig = px.scatter(
    points.values(),
    x="E",
    y="N",
    hover_data=["Name"],
    color="Massstabfaktor",
    # size="Massstabfaktor",
    title=f"Massstabsfaktor",
)

fig.show()

<IPython.core.display.Javascript object>

In [63]:
fig = px.scatter(
    points.values(),
    x="E",
    y="N",
    hover_data=["Name"],
    color="Höhenverzerrungsfaktor",
    size="Height",
    title=f"Höhenverzerrungsfaktor",
)

fig.show()

<IPython.core.display.Javascript object>

In [64]:
df = pd.DataFrame(points).transpose()
df.style.format(
    {
        "N": "{:.1f}",
        "E": "{:.1f}",
        "Height": "{:.2f}",
        "Längenfehler [mm]": "{:.3f}",
        "Gesamtfehler [mm]": "{:.3f}",
        "max. Distanz [m]": "{:.3f}",
    }
)

,Name,N,E,Height,Massstabfaktor,Höhenverzerrungsfaktor,Längenfehler [mm],Gesamtfehler [mm],max. Distanz [m]
Cadenazzo,Cadenazzo,1112721.7,2715783.3,208.00,1.000094,1.000033,93.606,60.999,163.937
Bern,Bern,1200000.0,2600000.0,553.60,1.000000,1.000087,0.000,-86.780,115.234
Muttenz,Muttenz,1264806.3,2615766.2,280.70,1.000052,1.000044,51.609,7.606,1314.791
Chessel,Chessel,1133230.0,2558128.3,378.60,1.000055,1.000059,54.784,-4.565,2190.424
Melano,Melano,1086300.0,2719998.3,281.70,1.000159,1.000044,158.859,114.699,87.185
Kraftwerk Untere Enge,Kraftwerk Untere Enge,1201114.0,2600259.5,500.00,1.000000,1.000078,0.015,-78.363,127.611
Bottinghofen am Bodensee,Bottinghofen am Bodensee,1278503.3,2733173.3,400.00,1.000076,1.000063,75.730,13.026,767.697
Zürich HB,Zürich HB,1248258.0,2682858.2,406.20,1.000029,1.000064,28.617,-35.058,285.240
Boncourt,Boncourt,1260400.0,2568683.3,444.40,1.000045,1.000070,44.829,-24.834,402.678
CERN,CERN,1121229.0,2493320.0,437.90,1.000076,1.000069,76.247,7.603,1315.343


<IPython.core.display.Javascript object>

### Höhenfehler wegen Erdkrümmung

In [76]:
fig = go.Figure()

# Create scatter trace of text labels
fig.add_trace(
    go.Scatter(
        x=[2.5, 2.5, 2.95, 3.05],
        y=[2.05, 2.3, 2.5, 2.25],
        text=["R", "R", "h0", "d"],
        mode="text",
    )
)
# Set axes properties
fig.update_xaxes(range=[1, 3.5], zeroline=False)
fig.update_yaxes(range=[1, 3])


# Add circles
fig.update_layout(
    shapes=[
        # unfilled circle
        dict(
            type="circle",
            xref="x",
            yref="y",
            x0=1,
            y0=1,
            x1=3,
            y1=3,
            line_color="LightSeaGreen",
        ),
        # tangent
        dict(
            type="line",
            xref="x",
            yref="y",
            x0=3,
            y0=1.25,
            x1=3,
            y1=2.75,
            line_color="LightSeaGreen",
        ),
        dict(
            type="line",
            xref="x",
            yref="y",
            x0=2,
            y0=2,
            x1=3,
            y1=2.5,
            line_color="LightSeaGreen",
        ),
        dict(
            type="line",
            xref="x",
            yref="y",
            x0=2,
            y0=2,
            x1=3,
            y1=2,
            line_color="LightSeaGreen",
        ),
    ]
)

<IPython.core.display.Javascript object>

In [80]:
print(
    f"Der Höhenfehler h0 für ein Projekt von {messung} m in einem lokalen kartesischen Koordinatensystem beträgt {h0} mm."
)

Der Höhenfehler h0 für ein Projekt von 1000 m in einem lokalen kartesischen Koordinatensystem beträgt 78.38445343077183 mm.


<IPython.core.display.Javascript object>

## Weiterentwicklungsideen 
Angabe von min/max Höhe, min/max N, Ausdehnung. Darauf basierend Berechnung von durchschnittlicher Verzerrung und Delta.